In [1]:
# importing libraries
import nltk
import re
import pandas as pd
# import numpy as np
# from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
# from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC  # SVM

In [2]:
# stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
# cv = CountVectorizer()
tfidfv = TfidfVectorizer()

In [3]:
from sklearn.datasets import fetch_20newsgroups

In [4]:
categories = ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware']

In [5]:
dataset_training = fetch_20newsgroups(subset='train', shuffle=True, categories=categories,)

In [6]:
twenty_news_data = fetch_20newsgroups()

In [8]:
dataset_training.__len__()

5

In [9]:
print("\n".join(dataset_training.data[0].split("\n")[:3]))

From: hades@coos.dartmouth.edu (Brian V. Hughes)
Subject: Re: New Apple Ergo-Mouse
Reply-To: hades@Dartmouth.Edu


In [7]:
labels = dataset_training.target
text = dataset_training.data
print(len(labels), len(text))

2823 2823


In [8]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
Labels = le.fit_transform(labels)
labels

array([4, 2, 3, ..., 4, 1, 2])

In [12]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [9]:
corpus = []
for i in range(len(text)):
    words = re.sub('[^a-zA-Z]', ' ', text[i])
    words = words.lower()
    words = nltk.word_tokenize(words)
    # words = nltk.word_tokenize(sentences[i])
    words = [lemmatizer.lemmatize(word) for word in words if word not in set(stopwords.words("english"))]
    words = ' '.join(words)
    corpus.append(words)

In [14]:
corpus.__len__()

2823

In [10]:
# Convert to TF-IDF format
vectorizer = TfidfVectorizer(max_df=0.5, min_df=2, stop_words='english', use_idf=True)
# X_train = vectorizer.fit_transform(data_train.data)

In [11]:
X = vectorizer.fit_transform(corpus).toarray()

In [12]:
X.shape

(2823, 16288)

In [13]:
Y = Labels

In [18]:
import numpy as np
from scipy.spatial.distance import pdist, squareform


In [19]:
def rdc(X, y, gamma):
    n_features = X.shape[1]
    pairwise_dists = squareform(pdist(X, 'euclidean'))

    K = np.exp(-gamma * pairwise_dists ** 2)
    C_xy = np.corrcoef(np.hstack([X, y.reshape(-1, 1)]).T)[-1, :-1]
    C_xx = np.corrcoef(X.T)

    rdc_scores = np.zeros(n_features)
    for i in range(n_features):
        rdc_scores[i] = np.sum((C_xy[i] - np.sum(C_xy) / n_features) * K[i]) / np.sqrt(
            np.sum((C_xx[i] - np.sum(C_xx) / n_features) ** 2) * np.sum(K[i] ** 2))
    return rdc_scores


In [ ]:
gamma = 0.1 # hyperparameter
rdc_scores = rdc(X, Y, gamma)


In [14]:
print(X.shape, Y.shape)

(2823, 16288) (2823,)


In [15]:
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, random_state = 0, test_size = 0.1)

# SVM Model

In [16]:
svc = SVC(kernel="linear")
model = svc.fit(xtrain, ytrain)

In [ ]:
dtct = model.predict(xtest)

In [ ]:
cm = confusion_matrix(ytest, dtct)
acuuracy = accuracy_score(ytest, dtct)
print(acuuracy)
print(cm)

0.9328621908127208
[[42  1  0  0  0]
 [ 0 54  1  0  0]
 [ 0  5 51  1  0]
 [ 0  2  2 62  5]
 [ 0  0  1  1 55]]


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


In [ ]:
import pickle
f = open('20_news_svm_classifier.pickle', 'wb')
pickle.dump(model, f)
f.close()

In [ ]:
import pickle
f = open('20_news_svm_classifier.pickle', 'rb')
classifier = pickle.load(f)
f.close()

In [ ]:
acrcy = accuracy_score(ytest, classifier.predict(xtest))
print(acrcy)

0.9328621908127208


# Naive Bayes Model

In [ ]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb_model = nb.fit(xtrain, ytrain)

In [ ]:
detect = nb_model.predict(xtest)

In [ ]:
c_m = confusion_matrix(ytest, detect)
accuracy = accuracy_score(ytest, detect)
print(accuracy)
print(c_m)

0.901060070671378
[[43  0  0  0  0]
 [ 0 51  1  2  1]
 [ 0  5 46  5  1]
 [ 0  2  1 62  6]
 [ 0  0  2  2 53]]
